In [ ]:
from time import sleep
from json import dumps
from kafka3 import KafkaProducer
import csv
import datetime as dt

In [ ]:
# hostip = "192.168.1.87"
hostip = ""

def publish_message(producer_instance, topic_name, data):
    try:
        value_bytes = bytes(data, encoding='utf-8')
        producer_instance.send(topic_name, value=value_bytes)
        print('Message published successfully. ' + data)
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        

In [ ]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=[f'{hostip}:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    

In [ ]:
if __name__ == '__main__':
    topic = 'Producer C'
    print('Publishing records..')
    producer = connect_kafka_producer()

    file_path = '/data/camera_event_C.csv'  # Path to your CSV file

    with open(file_path, mode='r') as file:
        csv_reader = csv.DictReader(file)
        for row in csv_reader:
            json_data = dumps(row)
            publish_message(producer, topic, json_data)
            sleep(1)

    producer.flush()